Installing Packages



In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 13.2MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0398548924f4d2723f574b443ce65ba40ca1abf02a9844573ed2d4457a182ef7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 204kB 4.4MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 317kB 13.4MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=f54db22648bd99f606c3d4734c472b070822030861d7ce3de154678f04c9485c
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
  Found existing installati

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=a086d4a76aa82112559fe47c0d965f500f749a7cccadba2dd40cc268f236d1cf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


Importing Libraries


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold 
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

#choose the same seed to assure that our model will be roproducible

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(2)

Reading Data 

In [ ]:
import pandas as pd
#We consider that our data is a csv file (2 columns : text and label)
#using pandas function (read_csv) to read the file
train=pd.read_csv()

feat_cols = "text"


#Verify the topic classes in the data


In [ ]:
train.label.unique()

array(['art', 'politics', 'health', 'tourism'], dtype=object)

In [ ]:
label_cols = ['art', 'politics', 'health', 'tourism']
train.head()
l=['art', 'politics', 'health', 'tourism']
# Get the numerical ids of column label
train['label']=train.label.astype('category')

Y = train.label.cat.codes
train['label']=Y
# Print initial shape
print(Y.shape)
from keras.utils import to_categorical
# One-hot encode the indexes
Y = to_categorical(Y)

# Check the new shape of the variable
print(Y.shape)

# Print the first 5 rows
print(Y[0:5])
for i in range(len(l)) :     
     train[l[i]] = Y[:,i]

(4000,)
(4000, 4)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


Using TensorFlow backend.


In [ ]:
#It is too simple to use the ClassificationModel from simpletransformes :
   #ClassificationModel('Architecture', 'model shortcut name', use_cuda=True,num_labels=4)
   #Architecture : Bert , Roberta , Xlnet , Xlm...
   #shortcut name models for Roberta : roberta-base , roberta-large .... 
   #more details on : https://huggingface.co/transformers/pretrained_models.html 
   

In [ ]:
#using KFOLD Cross Validation is important to test our model

In [ ]:
%%time
err=[]
y_pred_tot=[]

fold=StratifiedKFold(n_splits=5, shuffle=True, random_state=1997)
i=1
for train_index, test_index in fold.split(train,train['label']):
    train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
    model = ClassificationModel('roberta', 'roberta-base', use_cuda=True,num_labels=4, args={
                                                                         'train_batch_size':16,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 4,
                                                                         'max_seq_length': 128,
                                                                         'regression': False,
                                                                         'manual_seed': 1997,
                                                                         "learning_rate":2e-5,
                                                                         'weight_decay':0,
                                                                         "save_eval_checkpoints": True,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": True})
    model.train_model(train1_trn)
    raw_outputs_val = model.eval_model(train1_val)[1]
    raw_outputs_vals = softmax(raw_outputs_val,axis=1)
    print(f"Log_Loss: {log_loss(train1_val['label'], raw_outputs_vals)}")
    err.append(log_loss(train1_val['label'], raw_outputs_vals))


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.32932861055553075


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Log_Loss: 0.31324721513781695


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Log_Loss: 0.357522397101493


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Log_Loss: 0.39003183998007446


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Log_Loss: 0.35637871529928816
CPU times: user 11min 2s, sys: 4min 21s, total: 15min 23s
Wall time: 16min 7s


In [ ]:
print("Mean LogLoss: ",np.mean(err))

Mean LogLoss:  0.34930175561484067


In [ ]:
raw_outputs_vals

array([[9.9822301e-01, 3.4856689e-04, 3.8243082e-04, 1.0458552e-03],
       [9.9695909e-01, 1.1522240e-03, 5.9563853e-04, 1.2927916e-03],
       [9.9910539e-01, 2.3084633e-04, 2.5905663e-04, 4.0465154e-04],
       ...,
       [3.6545596e-04, 2.8826005e-04, 4.3145564e-04, 9.9891484e-01],
       [4.0789684e-03, 9.9224585e-01, 1.2752400e-03, 2.3997365e-03],
       [3.7382307e-04, 3.4797701e-04, 3.6257200e-04, 9.9891579e-01]],
      dtype=float32)

In [ ]:
for i in range(len(raw_outputs_vals)):
  for j in range(4):
       if(max(raw_outputs_vals[i])==raw_outputs_vals[i][j]):
         raw_outputs_vals[i][j]=1
       else :
         raw_outputs_vals[i][j]=0

In [ ]:
raw_outputs_vals

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

Test Our model


In [ ]:

pred = model.predict(['i want to travel to thailand'])[1]
preds = softmax(pred,axis=1)

In [ ]:
preds

array([[6.0461409e-04, 3.6119239e-04, 3.3729596e-04, 9.9869716e-01]],
      dtype=float32)

In [ ]:
#we create a function which calculate the maximum probability and detect the topic
#for example if we have 0.6 politics 0.1 art 0.15 health 0.15 tourism >>>> topic = politics 

In [ ]:
def estm(raw_outputs_vals):
 for i in range(len(raw_outputs_vals)):
   for j in range(4):
       if(max(raw_outputs_vals[i])==raw_outputs_vals[i][j]):
         raw_outputs_vals[i][j]=1
       else :
         raw_outputs_vals[i][j]=0
 return(raw_outputs_vals) 

In [ ]:
estm(preds)

array([[0., 0., 0., 1.]], dtype=float32)

In [ ]:
#our labels are :['art', 'politics', 'health', 'tourism']
#so that's correct ;)